## Scope of Variables

https://docs.julialang.org/en/v1/manual/variables-and-scoping/#scope-of-variables


The scope of a variable is the region of code within which a variable is visible. Variable scoping helps avoid variable naming conflicts.

There are two main types of scopes in Julia, global scope and local scope. The latter can be nested.

While we will not cover scope in full detail, some examples will be given so we get an intuitive sense of what is going on.

### Global Scope

* Each module introduces a new global scope, separate from the global scope of all other modules—there is no all-encompassing global scope. 
* Modules can introduce variables of other modules into their scope through the using or import statements or through qualified access using the dot-notation, i.e. each module is a so-called namespace as well as a first-class data structure associating names with values. 
*Note that while variable bindings can be read externally, they can only be changed within the module to which they belong

In [1]:
module A
    a = 1 # a global in A's scope
end

Main.A

In [2]:
module B
    module C
        c = 2
    end
    b = C.c    # can access the namespace of a nested global scope
               # through a qualified access
    import ..A # makes module A available
    d = A.a
end

Main.B

In [ ]:
module D
    b = a # errors as D's global scope is separate from A's
end

In [ ]:
module E
    import ..A # makes A available
    A.a = 2 # error's as we cannot modify data in other modules.
end

## Local Scope

While there are specific rules in Julia for what the expressiosn means based on where the assignment expression occurs, we will only look at this intuitively. 

Compare the following examples

In [3]:
function hello()
    x = "hello world"
    println(x)
end

hello (generic function with 1 method)

In [4]:
hello()

hello world


In [5]:
x #global scope

LoadError: UndefVarError: x not defined

Inside of the `hello` function, the assignment `x = "hello world"` causes `x` to be a new local variable in the function's scope. Since `x` is local , it doesn't matter if there is a global named `x` or not. See what happens when we define `x=123` before defining and calling `hello`.

In [6]:
x = 123 # global

123

In [7]:
function hello()
    x = "hello world"
    println(x)
end

hello (generic function with 1 method)

In [8]:
hello()

hello world


In [9]:
x

123

Since the x in `hello` is local, the value (or lack thereof) of the global `x` is unaffected by calling `hello`.

In [10]:
function sum_to(n)
    s = 0 # new local
    for i in 1:n
        s = s + i # assign existing local
    end
    return s # same local
end

sum_to (generic function with 1 method)

In [11]:
sum_to(10)

55

In [12]:
s #global

LoadError: UndefVarError: s not defined

In [13]:
s = 0
for i = 1:10
    s += i
end

In [14]:
s

55

A `for` loop or comprehension iteration variable is always a new variable:

In [15]:
function f()
    i = 0
    for i in 1:3
        #empty
    end
    return i
end

f (generic function with 1 method)

In [16]:
f()

0

However, it is occasionally useful to reuse an existing local variable as the iteration variable. This can be done conveniently by adding the keyword `outer`:

In [17]:
function f()
    i = 0
    for outer i in 1:3
        #empty
    end
    return i
end

f (generic function with 1 method)

In [18]:
f()

3

In [19]:
i

LoadError: UndefVarError: i not defined

Finally, we can force local variables to be usable by the global scope, using the keyword `global` : 

In [20]:
function f()
    global i = 0
    for outer i in 1:3
        #empty
    end
    return i
end

f (generic function with 1 method)

In [21]:
i

LoadError: UndefVarError: i not defined

In [22]:
f()

3

In [23]:
i

3

## Constants
A common use of variables is giving names to specific, unchanging values. Such variables are only assigned once. This intent can be conveyed to the compiler using the `const` keyword:

In [24]:
const e  = 2.71828182845904523536;
const pi = 3.14159265358979323846;

Multiple variables can be declared in a single `const` statement:

In [25]:
const a, b = 1, 2

(1, 2)

In [1]:
const x = 1.1

1.1

In [2]:
x

1.1

If a new value has a different type than the type of the constant then an error is thrown:

In [34]:
x = 1

LoadError: invalid redefinition of constant xs

If a new value has the same type as the constant then a warning is printed:

In [3]:
const y = 1.0

1.0

In [4]:
y = 2.0

2.0

### Performance tip : 
The const declaration should only be used in global scope on globals. It is difficult for the compiler to optimize code involving global variables, since their values (or even their types) might change at almost any time. If a global variable will not change, adding a `const` declaration solves this performance problem.

## Benchmarking tips

In the `Julia is fast` notebook, we saw the package `BenchmarkTools` and used its `@benchmark` macro.

In this notebook, we'll explore the importance of "interpolating" global variables when benchmarking functions.

We interpolate a global variable by throwing a `$` in front of it. For example, in `Julia is fast`, we benchmarked the `sum` function using `Vector` `A` via

```julia
@benchmark sum($A)
```

not

```julia
@benchmark sum(A)
```

Let's see if this can make a difference by examining the ratio in execution times of `sum($A)` and `sum(A)` for differently sized arrays `A`. 

#### Exercise 13.1

Call the `sum` function on a pseudo-randomly populated 1D array called `foo` of several lengths between 2 and 2^20 (~10^6). For each size of `foo`, determine the ratio of execution times for `sum(foo)` and `sum($foo)`. (To determine this ratio, use the minimum run times in each case.)

Plot the ratio of execution times for non-interpolated and interpolated `foo` in calls to `sum` versus the length of `foo`. Does interpolating `foo` seem to matter? If so, for what sizes of `foo`?

In [2]:
using BenchmarkTools, Plots
gr()

Plots.GRBackend()

In [3]:
store_ratios = []
foo = rand()
for i in 1:20
    foo = rand(2^i) 
    not_interpolated = @benchmark sum(foo)
    interpolated = @benchmark sum($foo)
    min_not_interpolated = minimum(not_interpolated.times)
    min_interpolated = minimum(interpolated.times)
    ratio = min_not_interpolated / min_interpolated
    append!(store_ratios, ratio)
end

In [ ]:
plot([i for i in 1:20], store_ratios, legend=false)
scatter!([i for i in 1:20], store_ratios)
xlabel!("The log of the length of `foo`")
ylabel!("Factor of performance increase")
title!("Performance increase from interpolating a global variable")

## Performance tips -- type stability

One way to optimize code in Julia is to ensure **type stability**. If the type(s) of some variables in a function are subject to change or ambiguity, the compiler cannot reason as well about those variables, and performance will take a hit. Conversely, we allow the compiler to optimize and generate more efficient machine code when we declare variables so that their types will be fixed throughout the function body.

#### Exercise 13.1

The below definition for `baz()` is not optimised. Copy it and define a new function `bar()` in the designated cell below and improve its performance!

In [41]:
function baz()
    s = rand()
    if s > 2/3
        return .666667
    elseif s > 1/3
        return 1//3    
    else
        return 0    
    end
end

baz (generic function with 1 method)

In [42]:
using BenchmarkTools
@benchmark baz()


BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  10.791 ns … 43.666 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.458 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.732 ns ±  1.197 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▆▆█▆▆                                                    
  ▁▂▃▅█████▆▃▃▃▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  10.8 ns         Histogram: frequency by time        16.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [43]:
function bar() :: Float64 # This may be enough!
    s = rand()
    if s > 2/3
        return .666667
    elseif s > 1/3
        return 0.33333 # Float
    else
        return 0.0 # Float
    end
end

bar (generic function with 1 method)

In [44]:
@benchmark bar()

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  3.041 ns … 24.500 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.167 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.232 ns ±  0.614 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁ ▇▇█▄ ▂▂ ▃                                     ▁ ▁    ▁  ▂
  ▅█▁████▁██▁█▁██▁█▁▇▆▁▆▁▅▆▁▅▁▄▅▁▅▁▆▄▁▄▁▄▄▁▅▁▄▅▁▅▁▆█▁█▁▇▇▁█▆ █
  3.04 ns      Histogram: log(frequency) by time      4.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.